<a href="https://colab.research.google.com/github/1NetoDev/curriculAI/blob/main/HabituAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Célula 1: Instalações e Configuração da API Key

# Instalações de bibliotecas Python
!pip install -q gradio google-generativeai

print("Instalações concluídas.")

# Configuração da API Key do Google AI
import os
from google.colab import userdata
import google.generativeai as genai

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY is None:
        raise ValueError("Chave 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab. Defina-a ou edite o código para desenvolvimento local.")

    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave da API do Google AI configurada com sucesso para o App de Hábitos!")
except Exception as e:
    print(f"ATENÇÃO: Erro ao configurar a chave da API do Google AI: {e}")
    print("A funcionalidade de IA não funcionará sem a chave da API.")
    GOOGLE_API_KEY = None

Instalações concluídas.
Chave da API do Google AI configurada com sucesso para o App de Hábitos!


In [ ]:
# Célula 2.1: Estado da Sessão e Constantes do Sistema

import json # Será usado mais tarde se a IA retornar JSON

# --- Estado da Sessão do Usuário (Simples, para o MVP) ---
usuario_objetivos_iniciais = ""
conversa_com_ia_para_habitos = []
plano_de_habitos_gerado = []
xp_total = 0
gold_total = 0
nivel_usuario = 1
id_habito_counter = 0

# --- Instruções de Sistema para a IA ---
INSTRUCAO_SISTEMA_INICIAL = """
Você é 'HabitMaster IGNITE', um Mestre de Jogo e Coach de Hábitos carismático e motivador.
Seu objetivo é ajudar o 'Aventureiro' (usuário) a transformar seus objetivos de vida em uma jornada épica de construção de hábitos.
Sua primeira interação é para entender os objetivos gerais do Aventureiro.
Faça de 1 a 3 perguntas abertas e encorajadoras para que ele possa elaborar sobre o que deseja alcançar ou melhorar em sua vida.
Seja breve, amigável e com um toque temático de RPG. Não sugira hábitos ainda. Apenas colete informações.
Exemplo de pergunta: "Saudações, nobre Aventureiro! Para que possamos forjar sua lenda, conte-me: quais grandes feitos você sonha em realizar ou quais masmorras de maus hábitos deseja conquistar?"
Finalize sua resposta com uma pergunta clara para o usuário.
"""

INSTRUCAO_SISTEMA_REFINAMENTO_E_SUGESTAO = """
Você é 'HabitMaster IGNITE'. O Aventureiro compartilhou seus objetivos e respondeu suas perguntas iniciais.
Agora, sua tarefa é:
1.  Analisar TODAS as respostas anteriores dele no histórico da conversa.
2.  Se as informações ainda forem muito vagas para criar hábitos concretos, faça MAIS UMA ou DUAS perguntas MUITO ESPECÍFICAS para clarificar ou detalhar um objetivo. Mantenha o tom de RPG.
3.  Se as informações forem suficientes, SUGIRA de 3 a 5 'Missões Iniciais' (hábitos).
    Para cada Missão/Hábito:
    - Dê um nome temático e divertido (Ex: 'Hidratação do Grifo', 'Passeio Élfico Matinal', 'Leitura dos Pergaminhos Antigos').
    - Defina uma META CLARA e ALCANÇAVEL para iniciantes (Ex: 'Beber 1L de água por dia', 'Caminhar 15 minutos ao ar livre', 'Ler 2 capítulos').
    - Atribua XP (entre 10 e 50, baseado na dificuldade percebida) e GOLD (entre 1 e 5).
    - Especifique a FREQUÊNCIA (Ex: 'Diário', '3x por semana', 'Seg-Sex').
4.  APRESENTE este plano como um pergaminho de missões. Use formatação Markdown consistente para facilitar o processamento:
    - Inicie a seção do plano com uma frase clara como "### PERGAMINHO DAS MISSÕES INICIAIS ###".
    - Cada missão deve começar com `**Missão: [Nome da Missão]**`.
    - Os detalhes devem ser itens de lista: `- Meta: [descrição]`, `- Frequência: [descrição]`, `- Recompensa: [XP] XP, [GOLD] Gold`.
5.  Após apresentar o plano (ou fazer mais perguntas), finalize sua resposta PERGUNTANDO ao Aventureiro se ele gostaria de iniciar com estas missões, se deseja ajustes, ou respondendo à sua última pergunta.

Exemplo de Saída de Plano (APENAS O PLANO, após as perguntas se houver):
```markdown
### PERGAMINHO DAS MISSÕES INICIAIS ###

**Missão: Hidratação do Grifo**
- Meta: Beber 1,5 litros de água
- Frequência: Diário
- Recompensa: 20 XP, 2 Gold

**Missão: Leitura dos Pergaminhos Antigos**
- Meta: Ler por 20 minutos
- Frequência: 3x por semana
- Recompensa: 35 XP, 3 Gold

**Missão: Forja da Mente (Meditação)**
- Meta: Meditar por 5 minutos
- Frequência: Diário
- Recompensa: 25 XP, 2 Gold

Pronto para embarcar nestas missões, ou gostaria de fazer algum ajuste neste pergaminho sagrado?
"""
print("Célula 2.1: Estado da Sessão e Constantes do Sistema DEFINIDOS.")

Célula 2.1: Estado da Sessão e Constantes do Sistema DEFINIDOS.


In [ ]:
# Célula 2.2: Função de Interação com a IA
import google.generativeai as genai # Necessário para genai.GenerativeModel

def chamar_ia_para_habitos(historico_conversa_atual, nova_mensagem_usuario_str, instrucao_sistema_especifica_str, nome_modelo_str="gemini-1.5-flash-latest"):
    print(f"DEBUG - chamar_ia_para_habitos: Nova mensagem do usuário: '{nova_mensagem_usuario_str}'")
    try:
        model = genai.GenerativeModel(
            model_name=nome_modelo_str,
            system_instruction=instrucao_sistema_especifica_str
        )
        prompt_construido_para_api = []
        for item in historico_conversa_atual:
            prompt_construido_para_api.append(f"{item['role'].title()}: {item['content']}")
        prompt_construido_para_api.append(f"User: {nova_mensagem_usuario_str}")
        prompt_completo_str = "\n\n".join(prompt_construido_para_api)

        print(f"DEBUG - chamar_ia_para_habitos: Prompt completo para IA (últimos 700 chars):\n...{prompt_completo_str[-700:]}")
        response = model.generate_content(prompt_completo_str)

        if not hasattr(response, 'text') or not response.text:
             print("DEBUG - chamar_ia_para_habitos: Resposta da IA não contém 'text' ou está vazia.")
             novo_historico_com_erro = historico_conversa_atual + [
                {"role": "user", "content": nova_mensagem_usuario_str},
                {"role": "model", "content": "ErroAPI_RespostaInesperadaOuVazia"}
             ]
             return "ErroAPI_RespostaInesperadaOuVazia", novo_historico_com_erro

        resposta_ia_texto_str = response.text
        resposta_ia_preview = resposta_ia_texto_str[:200].replace('\n', ' ')
        print(f"DEBUG - chamar_ia_para_habitos: Resposta da IA: '{resposta_ia_preview}...'")

        novo_historico_lista = historico_conversa_atual + [
            {"role": "user", "content": nova_mensagem_usuario_str},
            {"role": "model", "content": resposta_ia_texto_str}
        ]
        return resposta_ia_texto_str, novo_historico_lista

    except AttributeError as ae:
        msg_erro_api = "ErroAPI_ChaveNaoConfiguradaOuInvalida"
        if "'NoneType' object has no attribute 'send_request'" in str(ae) or \
           "configure() has not been called" in str(ae) or \
           "'GoogleGenerativeAI' object has no attribute 'default_transport'" in str(ae):
            print(f"DEBUG - chamar_ia_para_habitos: Erro crítico - API Key provavelmente não configurada: {ae}")
        else:
            print(f"DEBUG - chamar_ia_para_habitos: Erro de Atributo ao chamar a API Gemini: {ae}")
            msg_erro_api = f"ErroAPI_Atributo: {str(ae)}"
        novo_historico_com_erro_attr = historico_conversa_atual + [
            {"role": "user", "content": nova_mensagem_usuario_str},
            {"role": "model", "content": msg_erro_api}
        ]
        return msg_erro_api, novo_historico_com_erro_attr
    except Exception as e:
        print(f"DEBUG - chamar_ia_para_habitos: Erro GERAL ao chamar a API Gemini: {type(e).__name__} - {e}")
        error_details = getattr(e, 'message', str(e))
        if hasattr(e, 'response') and hasattr(e.response, 'text'): # Especifico para alguns erros de API HTTP
            error_details += f" | Detalhes da API: {e.response.text}"
        msg_erro_api_geral = f"ErroAPI_Comunicacao: {error_details}"
        novo_historico_com_erro_geral = historico_conversa_atual + [
            {"role": "user", "content": nova_mensagem_usuario_str},
            {"role": "model", "content": msg_erro_api_geral}
        ]
        return msg_erro_api_geral, novo_historico_com_erro_geral

print("Célula 2.2: Função de Interação com a IA DEFINIDA.")

Célula 2.2: Função de Interação com a IA DEFINIDA.


In [12]:
# Célula 2.3: Funções de Gamificação e Parseamento (AJUSTADA)

def marcar_habitos_concluidos_ui(lista_nomes_habitos_concluidos_da_ui):
    global xp_total, gold_total, plano_de_habitos_gerado, nivel_usuario # Acessa as globais da Célula 2.1

    print(f"DEBUG - marcar_habitos_concluidos_ui: Recebido da UI - {lista_nomes_habitos_concluidos_da_ui}")

    # Primeiro, desmarca todos os hábitos que não estão na lista da UI (se foram deselecionados)
    for habito_dict in plano_de_habitos_gerado:
        nome_habito_no_plano = habito_dict.get("nome", "")
        # Lógica de desmarcar se o hábito estava concluído mas não está mais na lista de selecionados
        if habito_dict.get("concluido", False) and nome_habito_no_plano not in lista_nomes_habitos_concluidos_da_ui:
            habito_dict["concluido"] = False
            xp_total -= habito_dict["xp"]
            gold_total -= habito_dict["gold"]
            if xp_total < 0: xp_total = 0
            if gold_total < 0: gold_total = 0
            print(f"DEBUG: Hábito '{habito_dict['nome']}' DESMARCADO via UI. Pontos revertidos.")

    # Agora, marca os hábitos que estão na lista da UI e ainda não foram marcados como concluídos no backend
    for nome_habito_concluido_ui in lista_nomes_habitos_concluidos_da_ui:
        habito_alvo = None
        for habito_dict_interno in plano_de_habitos_gerado:
            if habito_dict_interno.get("nome") == nome_habito_concluido_ui:
                habito_alvo = habito_dict_interno
                break

        if habito_alvo and not habito_alvo.get("concluido", False): # Só concede se não estava concluído
            habito_alvo["concluido"] = True
            xp_total += habito_alvo["xp"]
            gold_total += habito_alvo["gold"]
            print(f"DEBUG: Hábito '{habito_alvo['nome']}' MARCADO como concluído via UI! +{habito_alvo['xp']} XP, +{habito_alvo['gold']} Gold.")

            xp_para_proximo_nivel = nivel_usuario * 100
            if xp_total >= xp_para_proximo_nivel:
                nivel_usuario += 1
                # xp_total -= xp_para_proximo_nivel # Opcional: resetar XP para o próximo nível
                print(f"DEBUG: LEVEL UP! Você alcançou o Nível {nivel_usuario}!")
        elif habito_alvo and habito_alvo.get("concluido", False):
             print(f"DEBUG: Hábito '{nome_habito_concluido_ui}' já estava marcado como concluído no backend.")
        elif not habito_alvo:
            print(f"DEBUG: Hábito com nome '{nome_habito_concluido_ui}' (da UI) não encontrado no plano interno.")

    return get_estado_jogo_atualizado()


def get_estado_jogo_atualizado():
    global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
    # Para o CheckboxGroup, precisamos de uma lista de nomes e uma lista dos que estão concluídos
    nomes_habitos = [h.get("nome", f"Missão ID {h.get('id')}") for h in plano_de_habitos_gerado]
    habitos_concluidos_nomes = [h.get("nome") for h in plano_de_habitos_gerado if h.get("concluido", False)]

    return {
        "xp": xp_total,
        "gold": gold_total,
        "nivel": nivel_usuario,
        "habitos_detalhados": plano_de_habitos_gerado, # Lista completa para referência interna
        "nomes_habitos_para_checkboxgroup": nomes_habitos,
        "habitos_concluidos_para_checkboxgroup": habitos_concluidos_nomes
    }

def tentar_parsear_plano_de_habitos_da_ia(texto_resposta_ia_str):
    global plano_de_habitos_gerado, id_habito_counter

    habitos_extraidos_lista = []
    if "### PERGAMINHO DAS MISSÕES INICIAIS ###" not in texto_resposta_ia_str:
        print("DEBUG - tentar_parsear: Marcador de início do plano não encontrado.")
        return False

    linhas = texto_resposta_ia_str.split('\n')
    missao_atual_nome = None
    detalhes_habito_atual = {}

    # Resetar o contador de ID se um novo plano está sendo parseado
    # para evitar IDs muito grandes se o usuário gerar múltiplos planos na mesma sessão
    # Ou, melhor ainda, resetar o plano_de_habitos_gerado antes de popular
    plano_de_habitos_gerado = [] # Limpa o plano anterior antes de parsear um novo
    id_habito_counter = 0 # Reseta o contador de ID


    for linha_str in linhas:
        linha_strip = linha_str.strip()

        if linha_strip.startswith("**Missão:"):
            if missao_atual_nome and 'meta' in detalhes_habito_atual:
                detalhes_habito_atual['id'] = id_habito_counter
                id_habito_counter += 1
                detalhes_habito_atual['concluido'] = False
                detalhes_habito_atual.setdefault('frequencia', 'Diário')
                detalhes_habito_atual.setdefault('xp', 20)
                detalhes_habito_atual.setdefault('gold', 2)
                habitos_extraidos_lista.append(detalhes_habito_atual)

            missao_atual_nome = linha_strip.replace("**Missão:", "").replace("**", "").strip()
            detalhes_habito_atual = {"nome": missao_atual_nome}

        elif missao_atual_nome and linha_strip.startswith("- Meta:"):
            detalhes_habito_atual['meta'] = linha_strip.replace("- Meta:", "").strip()
        elif missao_atual_nome and linha_strip.startswith("- Frequência:"):
            detalhes_habito_atual['frequencia'] = linha_strip.replace("- Frequência:", "").strip()
        elif missao_atual_nome and linha_strip.startswith("- Recompensa:"):
            recompensa_str = linha_strip.replace("- Recompensa:", "").strip()
            try:
                partes_recompensa = recompensa_str.lower().split(',')
                xp_str = partes_recompensa[0].replace("xp", "").strip()
                detalhes_habito_atual['xp'] = int(xp_str)
                if len(partes_recompensa) > 1:
                    gold_str = partes_recompensa[1].replace("gold", "").strip()
                    detalhes_habito_atual['gold'] = int(gold_str)
                else:
                    detalhes_habito_atual['gold'] = int(int(xp_str) / 10) if int(int(xp_str) / 10) > 0 else 1
            except ValueError:
                print(f"DEBUG - tentar_parsear: Erro ao parsear XP/Gold para '{missao_atual_nome}'. Usando defaults.")
                detalhes_habito_atual['xp'] = 20
                detalhes_habito_atual['gold'] = 2

    if missao_atual_nome and 'meta' in detalhes_habito_atual and not any(d.get('nome') == missao_atual_nome for d in habitos_extraidos_lista):
        detalhes_habito_atual['id'] = id_habito_counter
        id_habito_counter += 1
        detalhes_habito_atual['concluido'] = False
        detalhes_habito_atual.setdefault('frequencia', 'Diário')
        detalhes_habito_atual.setdefault('xp', 20)
        detalhes_habito_atual.setdefault('gold', 2)
        habitos_extraidos_lista.append(detalhes_habito_atual)

    if habitos_extraidos_lista:
        plano_de_habitos_gerado = habitos_extraidos_lista
        print(f"DEBUG - tentar_parsear: Plano de hábitos ATUALIZADO da IA: {plano_de_habitos_gerado}")
        return True
    else:
        print("DEBUG - tentar_parsear: Nenhum hábito concreto foi parseado da resposta da IA.")
        return False

def resetar_jogo_e_conversa():
    global conversa_com_ia_para_habitos, plano_de_habitos_gerado, xp_total, gold_total, nivel_usuario, id_habito_counter
    conversa_com_ia_para_habitos = []
    plano_de_habitos_gerado = []
    xp_total = 0
    gold_total = 0
    nivel_usuario = 1
    id_habito_counter = 0 # Importante resetar o contador de ID também
    print("DEBUG: Jogo e conversa resetados.")
    return get_estado_jogo_atualizado()

print("Célula 2.3: Funções de Gamificação e Parseamento (AJUSTADAS) DEFINIDAS.")

Célula 2.3: Funções de Gamificação e Parseamento (AJUSTADAS) DEFINIDAS.


In [16]:
# Célula 3: Definição e Lançamento da Interface Gradio (CORRIGIDA - gr.Box para gr.Group)

import gradio as gr
# import os # Não é mais necessário aqui para o fluxo principal

# --- Definição do Tema ---
custom_theme = gr.themes.Base(
    primary_hue=gr.themes.colors.green,
    secondary_hue=gr.themes.colors.lime,
    neutral_hue=gr.themes.colors.gray,
    font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
)

css_custom = """
body { background-color: #121212 !important; color: #E0E0E0 !important; }
.gradio-container { background-color: #121212 !important; color: #E0E0E0 !important; }
.gr-input { background-color: #2C2C2C !important; color: #E0E0E0 !important; border-color: #4A4A4A !important; }
.gr-input::placeholder { color: #757575 !important; }
.gr-button.gr-button-primary { background-color: #32CD32 !important; color: white !important; border-color: #32CD32 !important;}
.gr-markdown h1, .gr-markdown h2, .gr-markdown h3 { color: #E0E0E0 !important; }
.chat-message.user-message { text-align: right; margin-left: 20%; margin-right: 5px; background-color: #2E7D32; padding: 8px; border-radius: 10px 10px 0 10px; margin-bottom: 5px;}
.chat-message.user-message strong { color: #C8E6C9; }
.chat-message.model-message { text-align: left; margin-right: 20%; background-color: #2a2a2a; border-left: 3px solid #32CD32; padding: 10px; margin-bottom:10px; border-radius:5px; }
.chat-message.model-message strong.model-name { color: #32CD32; }
.mission-details { color: #AAAAAA !important; font-size: 0.9em; }
.status-section { padding: 10px; background-color: #1E1E1E; border-radius: 5px; margin-bottom:15px;} /* Estilo para gr.Group */
.status-section .gr-markdown h3 { margin-top:0; color: #32CD32 !important;} /* Estilo para h3 dentro do grupo */
.gr-checkboxgroup label span {color: #E0E0E0 !important}
.gr-checkboxgroup .gr-input-label span {color: #E0E0E0 !important}
.gr-checkboxgroup .gr-check-radio span {color: #C0C0C0 !important}
#dashboard_habitos ul li { background-color: #252525 !important; border-radius: 4px !important; margin-bottom: 15px !important; padding: 8px !important; list-style-type: none; }
#dashboard_habitos ul li strong { color: #E0E0E0 !important; }
#dashboard_habitos ul li small { color: #AAAAAA !important; }
"""

with gr.Blocks(theme=custom_theme, title="HabitMaster IGNITE", css=css_custom) as demo:
    estado_conversa_hist = gr.State([])
    estado_jogo_ui = gr.State({"xp": 0, "gold": 0, "nivel": 1, "habitos_detalhados": [], "nomes_habitos_para_checkboxgroup": [], "habitos_concluidos_para_checkboxgroup": []})

    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1 style="color: #32CD32; font-family: 'Georgia', serif; text-shadow: 1px 1px #000;">⚔️ HabitMaster IGNITE ⚔️</h1>
            <p style="color: #B0B0B0; font-size: 1.1em;">Forje sua Lenda, um Hábito de Cada Vez!</p>
        </div>
        """
    )

    with gr.Tabs() as tabs:
        with gr.TabItem("💬 Chat com HabitMaster / 🔥 Missões", id="chat_missoes"):
            with gr.Row():
                with gr.Column(scale=2):
                    output_conversa_ia = gr.Markdown(elem_id="output_conversa_markdown")
                    input_usuario_chat = gr.Textbox(label="Sua Resposta:", placeholder="Digite aqui...", lines=2, show_label=False)
                    btn_enviar_mensagem = gr.Button("▶️ Enviar Mensagem", variant="primary")

                with gr.Column(scale=1):
                    with gr.Group(elem_classes="status-section"): # CORRIGIDO: gr.Box para gr.Group
                        gr.Markdown("### 📜 Seu Pergaminho de Herói 📜")
                        nivel_display = gr.Markdown(f"**Nível:** 1")
                        xp_display = gr.Markdown(f"**XP:** 0")
                        gold_display = gr.Markdown(f"**Gold:** 0")

                    with gr.Group(elem_classes="status-section"): # CORRIGIDO: gr.Box para gr.Group
                        gr.Markdown("### 🔥 Missões Ativas 🔥")
                        checkbox_habitos_ui = gr.CheckboxGroup(
                            label="Marque as missões concluídas:",
                            choices=[],
                            value=[],
                            interactive=True
                        )
                        detalhes_habitos_display_md = gr.Markdown("", elem_id="detalhes_habitos_display")


            def atualizar_display_habitos_e_status_jogo(estado_jogo_atualizado_dict):
                novas_choices_checkbox = estado_jogo_atualizado_dict.get("nomes_habitos_para_checkboxgroup", [])
                novos_concluidos_checkbox = estado_jogo_atualizado_dict.get("habitos_concluidos_para_checkboxgroup", [])

                detalhes_md = ""
                if estado_jogo_atualizado_dict.get("habitos_detalhados"):
                    detalhes_md = "<ul class='mission-list'>" # Adicionada classe para CSS se necessário
                    for habito in estado_jogo_atualizado_dict["habitos_detalhados"]:
                        status_habito_emoji = "✅" if habito.get('concluido', False) else "⏳"
                        detalhes_md += (
                            f"<li class='mission-item'>" # Adicionada classe para CSS se necessário
                            f"{status_habito_emoji} <strong>{habito.get('nome', 'N/A')}</strong><br>"
                            f"<small class='mission-details'>"
                            f"  🎯 Meta: {habito.get('meta', 'N/A')} | 🗓️ Frequência: {habito.get('frequencia', 'N/A')}<br>"
                            f"  ✨ Recompensa: {habito.get('xp', 0)} XP, {habito.get('gold', 0)} Gold"
                            f"</small></li>"
                        )
                    detalhes_md += "</ul>"
                else:
                    detalhes_md = "Nenhuma missão ativa no momento."

                return (
                    gr.update(choices=novas_choices_checkbox, value=novos_concluidos_checkbox),
                    detalhes_md,
                    f"**Nível:** {estado_jogo_atualizado_dict['nivel']}",
                    f"**XP:** {estado_jogo_atualizado_dict['xp']}",
                    f"**Gold:** {estado_jogo_atualizado_dict['gold']}",
                    estado_jogo_atualizado_dict
                )

            def interagir_com_habitmaster_ui(mensagem_usuario_atual_str, historico_atual_gr_lista, estado_jogo_atual_gr_dict):
                global conversa_com_ia_para_habitos
                conversa_com_ia_para_habitos = list(historico_atual_gr_lista)

                global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
                xp_total = estado_jogo_atual_gr_dict["xp"]
                gold_total = estado_jogo_atual_gr_dict["gold"]
                nivel_usuario = estado_jogo_atual_gr_dict["nivel"]
                plano_de_habitos_gerado = list(estado_jogo_atual_gr_dict.get("habitos_detalhados", []))

                instrucao_usada = INSTRUCAO_SISTEMA_INICIAL
                if len(conversa_com_ia_para_habitos) > 0:
                    instrucao_usada = INSTRUCAO_SISTEMA_REFINAMENTO_E_SUGESTAO

                resposta_ia, novo_historico = chamar_ia_para_habitos(conversa_com_ia_para_habitos, mensagem_usuario_atual_str, instrucao_usada)
                conversa_com_ia_para_habitos = novo_historico

                conversa_formatada_md = ""
                for item_idx, item in enumerate(conversa_com_ia_para_habitos):
                    msg_id = f"msg-{item['role']}-{item_idx}"
                    if item["role"] == "user":
                        conversa_formatada_md += f"<div id='{msg_id}' class='chat-message user-message'><strong>Aventureiro:</strong> {item['content']}</div>\n"
                    elif item["role"] == "model":
                        conversa_formatada_md += f"<div id='{msg_id}' class='chat-message model-message'><strong class='model-name'>HabitMaster IGNITE:</strong><br>{item['content']}</div>\n"

                plano_foi_parseado_com_sucesso = False
                if not (isinstance(resposta_ia, str) and resposta_ia.startswith("ErroAPI_")):
                    if INSTRUCAO_SISTEMA_REFINAMENTO_E_SUGESTAO == instrucao_usada or "PERGAMINHO DAS MISSÕES INICIAIS" in resposta_ia:
                         plano_foi_parseado_com_sucesso = tentar_parsear_plano_de_habitos_da_ia(resposta_ia)

                estado_jogo_final_dict = get_estado_jogo_atualizado()
                checkbox_update, detalhes_md_update, nivel_md, xp_md, gold_md, _ = atualizar_display_habitos_e_status_jogo(estado_jogo_final_dict)

                return (
                    conversa_formatada_md,
                    conversa_com_ia_para_habitos,
                    checkbox_update,
                    detalhes_md_update,
                    nivel_md,
                    xp_md,
                    gold_md,
                    estado_jogo_final_dict
                )

            def checkbox_mudou_ui(lista_habitos_selecionados_ui, estado_jogo_atual_gr_dict):
                global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
                xp_total = estado_jogo_atual_gr_dict["xp"]
                gold_total = estado_jogo_atual_gr_dict["gold"]
                nivel_usuario = estado_jogo_atual_gr_dict["nivel"]
                plano_de_habitos_gerado = list(estado_jogo_atual_gr_dict.get("habitos_detalhados", []))

                print(f"DEBUG UI - checkbox_mudou_ui: Selecionados: {lista_habitos_selecionados_ui}")
                estado_jogo_apos_marcar = marcar_habitos_concluidos_ui(lista_habitos_selecionados_ui)

                checkbox_update, detalhes_md_update, nivel_md, xp_md, gold_md, _ = atualizar_display_habitos_e_status_jogo(estado_jogo_apos_marcar)
                return checkbox_update, detalhes_md_update, nivel_md, xp_md, gold_md, estado_jogo_apos_marcar

            checkbox_habitos_ui.change(
                fn=checkbox_mudou_ui,
                inputs=[checkbox_habitos_ui, estado_jogo_ui],
                outputs=[checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display, xp_display, gold_display, estado_jogo_ui]
            )

            btn_enviar_mensagem.click(
                fn=interagir_com_habitmaster_ui,
                inputs=[input_usuario_chat, estado_conversa_hist, estado_jogo_ui],
                outputs=[output_conversa_ia, estado_conversa_hist, checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display, xp_display, gold_display, estado_jogo_ui]
            ).then(lambda: gr.update(value=""), outputs=input_usuario_chat)

            def iniciar_jornada_interface():
                estado_resetado_dict = resetar_jogo_e_conversa()
                primeira_pergunta_ia, historico_inicial = chamar_ia_para_habitos([], "Olá, HabitMaster!", INSTRUCAO_SISTEMA_INICIAL)

                global conversa_com_ia_para_habitos
                conversa_com_ia_para_habitos = historico_inicial

                conversa_formatada_md = ""
                if not (isinstance(primeira_pergunta_ia, str) and primeira_pergunta_ia.startswith("ErroAPI_")):
                     conversa_formatada_md = f"<div class='chat-message model-message'><strong class='model-name'>HabitMaster IGNITE:</strong><br>{primeira_pergunta_ia}</div>\n"
                else:
                    conversa_formatada_md = f"<p style='color: red;'><strong>Erro ao iniciar:</strong> {primeira_pergunta_ia}</p>"

                checkbox_update, detalhes_md_update, nivel_md, xp_md, gold_md, _ = atualizar_display_habitos_e_status_jogo(estado_resetado_dict)

                return (
                    conversa_formatada_md,
                    conversa_com_ia_para_habitos,
                    checkbox_update,
                    detalhes_md_update,
                    nivel_md,
                    xp_md,
                    gold_md,
                    estado_resetado_dict
                )

            demo.load(
                fn=iniciar_jornada_interface,
                inputs=None,
                outputs=[output_conversa_ia, estado_conversa_hist, checkbox_habitos_ui, detalhes_habitos_display_md, nivel_display, xp_display, gold_display, estado_jogo_ui]
            )

        with gr.TabItem("🏰 Loja do Aventureiro", id="loja"):
            gr.Markdown(
                """
                ## 🛍️ Empório do Mestre dos Hábitos 🛍️
                <p style="font-size: 1.1em; color: #B0B0B0;">Saudações, Aventureiro!</p>
                <p>Este é o local onde, em breve, você poderá trocar seu suado <strong>XP</strong> e reluzente <strong>Gold</strong> por recompensas épicas, melhorias para sua jornada e talvez alguns artefatos místicos para auxiliar na sua conquista de hábitos!</p>
                <p><em>Atualmente, o Empório está sendo abastecido com os melhores itens de todo o reino... Volte em breve!</em></p>
                <p style="text-align:center; font-size:1.5em; margin-top: 20px;">🚧 EM CONSTRUÇÃO 🚧</p>
                """
            )
            # Você pode adicionar uma imagem aqui se quiser, como antes.
            # gr.Image("URL_DA_IMAGEM_BAU", label="Baú do Tesouro (Em Breve!)", ...)

    gr.Markdown(
        """
        ---
        <p style="text-align:center; font-size:0.9em; color:#757575;">
        Uma criação da Imersão IA Alura com HabitMaster IGNITE
        </p>
        """
    )

# Lançamento da interface
print("Iniciando a interface Gradio do HabitMaster IGNITE...")
demo.launch(debug=True, share=True)

Iniciando a interface Gradio do HabitMaster IGNITE...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://11a928f5032acb0e49.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG: Jogo e conversa resetados.
DEBUG - chamar_ia_para_habitos: Nova mensagem do usuário: 'Olá, HabitMaster!'
DEBUG - chamar_ia_para_habitos: Prompt completo para IA (últimos 700 chars):
...User: Olá, HabitMaster!
DEBUG - chamar_ia_para_habitos: Resposta da IA: 'Saudações, nobre Aventureiro!  Preparado para embarcar em uma jornada épica de autoaperfeiçoamento?  Para que possamos forjar sua lenda pessoal, conte-me:  1. Quais são os três maiores tesouros que vo...'
DEBUG: Jogo e conversa resetados.
DEBUG - chamar_ia_para_habitos: Nova mensagem do usuário: 'Olá, HabitMaster!'
DEBUG - chamar_ia_para_habitos: Prompt completo para IA (últimos 700 chars):
...User: Olá, HabitMaster!
DEBUG - chamar_ia_para_habitos: Resposta da IA: 'Saudações, nobre Aventureiro!  Prepare-se para uma jornada épica rumo à auto-realização! Antes de forjarmos sua arma definitiva – um sistema de hábitos imbatível – preciso entender sua visão. Conte-me...'
DEBUG: Jogo e conversa resetados.
DEBUG - chamar_ia_para_hab